In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
# Path to the dataset directory
dataset_dir = 'C:\\Users\\notif\\Code Clause Internship\\Gender and Age Classification with Image Recognization\\UTKFace'

# Create empty lists for images and labels
images = []
gender_labels = []
age_labels = []

# Iterate over the dataset folder structure
for root, dirs, files in os.walk(dataset_dir):
    for file in files:
        # Get the file path and labels from the file name
        image_path = os.path.join(root, file)
        age, gender = file.split('_')[:2]

        # Load and preprocess the image
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (128, 128))
        image = image.astype('float32') / 255.0

        # Append the preprocessed image and labels
        images.append(image)
        gender_labels.append(int(gender))
        age_labels.append(int(age))

In [3]:
# Convert the lists to NumPy arrays
X = np.array(images)
y_gender = np.array(gender_labels)
y_age = np.array(age_labels)

# Split the dataset into training and testing sets
X_train, X_test, y_gender_train, y_gender_test, y_age_train, y_age_test = train_test_split(
    X, y_gender, y_age, test_size=0.2, random_state=42)

In [4]:
# Gender Detection Model
gender_model = Sequential()
gender_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
gender_model.add(MaxPooling2D(pool_size=(2, 2)))
gender_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
gender_model.add(MaxPooling2D(pool_size=(2, 2)))
gender_model.add(Flatten())
gender_model.add(Dense(128, activation='relu'))
gender_model.add(Dense(1, activation='sigmoid'))

In [5]:
# Age Estimation Model
age_model = Sequential()
age_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 1)))
age_model.add(MaxPooling2D(pool_size=(2, 2)))
age_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
age_model.add(MaxPooling2D(pool_size=(2, 2)))
age_model.add(Flatten())
age_model.add(Dense(128, activation='relu'))
age_model.add(Dense(1, activation='linear'))

In [6]:
# Compile the models
gender_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
age_model.compile(loss='mean_absolute_error', optimizer='adam')

# Reshape the input data for training
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)

# Train the models
gender_model.fit(X_train, y_gender_train, batch_size=128, epochs=10, verbose=1)
age_model.fit(X_train, y_age_train, batch_size=128, epochs=10, verbose=1)


Epoch 1/10
149/149 [==============================] - 139s 753ms/step - loss: 0.5519 - accuracy: 0.7185
Epoch 2/10
149/149 [==============================] - 117s 783ms/step - loss: 0.3599 - accuracy: 0.8428
Epoch 3/10
149/149 [==============================] - 92s 618ms/step - loss: 0.3076 - accuracy: 0.8679
Epoch 4/10
149/149 [==============================] - 93s 624ms/step - loss: 0.2801 - accuracy: 0.8819
Epoch 5/10
149/149 [==============================] - 99s 666ms/step - loss: 0.2519 - accuracy: 0.8935
Epoch 6/10
149/149 [==============================] - 109s 728ms/step - loss: 0.2264 - accuracy: 0.9051
Epoch 7/10
149/149 [==============================] - 112s 752ms/step - loss: 0.2043 - accuracy: 0.9156
Epoch 8/10
149/149 [==============================] - 119s 799ms/step - loss: 0.1852 - accuracy: 0.9272
Epoch 9/10
149/149 [==============================] - 113s 758ms/step - loss: 0.1689 - accuracy: 0.9316
Epoch 10/10
149/149 [==============================] - 112s 751ms/s

In [7]:
# Evaluate the gender model
y_gender_pred = np.round(gender_model.predict(X_test))
gender_accuracy = accuracy_score(y_gender_test, y_gender_pred)
print("Gender Accuracy:", gender_accuracy)

149/149 [==============================] - 10s 55ms/step
Gender Accuracy: 0.8907633909742725


In [8]:
# Evaluate the age model
y_age_pred = np.round(age_model.predict(X_test))
age_accuracy = accuracy_score(y_age_test, y_age_pred)
print("Age Accuracy:", age_accuracy)

149/149 [==============================] - 10s 61ms/step
Age Accuracy: 0.07612821594264024


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [10]:
# Gender Detection Model (CNN)
gender_model = Sequential()
# Architecture same as before, please refer to Step 4 of the previous code.

# Age Estimation Model (Random Forest)
age_model_rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Age Estimation Model (XGBoost)
age_model_xgb = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

In [ ]:
# Train the gender detection model (CNN)
# Please refer to Step 5 of the previous code.
# Train the age estimation models (Random Forest and XGBoost)
# Flatten the input data

X_train_flattened = X_train.reshape(X_train.shape[0], -1)

# Train the Random Forest model
age_model_rf.fit(X_train_flattened, y_age_train)

# Train the XGBoost model
age_model_xgb.fit(X_train_flattened, y_age_train)

In [ ]:
# Preprocess the testing data
X_test_flattened = X_test.reshape(X_test.shape[0], -1)

# Predict the gender using the CNN model
y_gender_pred = np.round(gender_model.predict(X_test))
gender_accuracy = accuracy_score(y_gender_test, y_gender_pred)
print("Gender Accuracy:", gender_accuracy)

# Predict the age using the Random Forest model
y_age_pred_rf = age_model_rf.predict(X_test_flattened)
age_mae_rf = mean_absolute_error(y_age_test, y_age_pred_rf)
print("Age MAE (Random Forest):", age_mae_rf)

# Predict the age using the XGBoost model
X_test_xgb = xgb.DMatrix(X_test_flattened)
y_age_pred_xgb = age_model_xgb.predict(X_test_xgb)
age_mae_xgb = mean_absolute_error(y_age_test, y_age_pred_xgb)
print("Age MAE (XGBoost):", age_mae_xgb)